# Chatbot using RNN
### Northwestern University - Fall 2017
### Student: Danilo Neves Ribeiro
### E-mail: daniloribeiro2021@u.northwestern.edu

# Overview

The idea of the project will be to implement a simple chat-bot using Recurrent neural networks. 


# Implementation Architecture
Here I use a RNN to train on the data set.

## Software requirements
- Python 3.6.2
- Numpy
- TensorFlow 

# Code solution

In [31]:
# IMPORTS

import os
import numpy as np
import re
import codecs
from nmt import nmt

In [32]:
# GLOBAL VARIABLES AND PARAMS

# encoding and decoding
TRAIN_END_PATH = os.path.join('data', 'train.enc')
TRAIN_DEC_PATH = os.path.join('data', 'train.dec')
TEST_END_PATH = os.path.join('data', 'test.enc')
TEST_DEC_PATH = os.path.join('data', 'test.dec')

# vocabulary
VOCAB_ENC_PATH = os.path.join('data', 'vocab.enc')
VOCAB_DEC_PATH = os.path.join('data', 'vocab.dec')
MAX_VOCAB_SIZE = 20000

# data utils
SPLIT_REGEX = re.compile("[\w]+|[.,!?\"':;)(]")
UNKNOWEN_TOKEN = "<unk>"
START_TOKEN = "<s>"
END_TOKEN = "</s>"
INIT_VOCAB = [UNKNOWEN_TOKEN, START_TOKEN, END_TOKEN]


In [33]:
# TOKENIZER

def tokenize(sentense):
    return re.findall(SPLIT_REGEX, sentense)

In [34]:
# CREATING VOCABULARY

def create_vocab(data_path, vocab_path):
    vocab = {}    
    # only creates new file is file doesn't exist
    if os.path.exists(vocab_path):
       print("file ", vocab_path, " already exists") 
    else:
        with open(data_path, 'r') as data_file:
            for line in data_file:
                tokens = tokenize(line)
                for token in tokens:
                    if token not in vocab:
                        vocab[token] = 1
                    else:
                        vocab[token] += 1
        # use the default tokens as initial vocabulity words
        vocab_list = INIT_VOCAB + sorted(vocab, key=vocab.get, reverse=True)
        # trim vocabulary
        vocab_list = vocab_list[:MAX_VOCAB_SIZE]
        print("final vacabulary size for ", data_path, " = ", len(vocab_list))
        # save to file
        with codecs.open(vocab_path, 'w', encoding='utf-8') as vocab_file:
            for word in vocab_list:
                vocab_file.write(word + "\n")
            

In [35]:
# PREPARING DATA

def prepare_data():
        create_vocab(TRAIN_END_PATH, VOCAB_ENC_PATH)
        create_vocab(TRAIN_DEC_PATH, VOCAB_DEC_PATH)
        
prepare_data()

final vacabulary size for  data\train.enc  =  20000
final vacabulary size for  data\train.dec  =  20000


# Evaluation



In [2]:
# testing
def evaluate():
    

In [ ]:
# interface


# Conclusion


## Challenges